# Analysis of Miracidia on Reduced Dataset

### Library QC

All samples except Liv-03 had a small peak at 135bp to 165bp. Liv-03 had no peak.

### Get reference genome and annotations

In [2]:
!mkdir genome_reduced
%cd genome_reduced
!wget -nc -O - https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.genomic.fa.gz | \
    gzip -f -d > genome.fa

!wget -O - https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.canonical_geneset.gtf.gz | \
    gzip -f -d > annotations.gtf

%cd ..

/data/users/corwinbm5021/Sm_Mira_IvT/notebook_reduced/genome_reduced
--2024-12-12 13:54:54--  https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.genomic.fa.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116797085 (111M) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 111.39M   603KB/s    in 3m 10s  

2024-12-12 13:58:04 (601 KB/s) - written to stdout [116797085/116797085]

--2024-12-12 13:58:04--  https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.canonical_geneset.gtf.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:443... connected.
HTTP re

### Run fastqc on fastq files

In [3]:
!mkdir fastq_reduced
!mkdir fastq_reduced/qc
!fastqc -t 32 /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/*.fastq.gz -o fastq_reduced/qc

application/gzip
application/gzip
Started analysis of Int-01_S42_L005_R1_001.fastq.gz
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
Started analysis of Int-01_S42_L005_R2_001.fastq.gz
Started analysis of Int-02_S43_L005_R1_001.fastq.gz
Started analysis of Int-02_S43_L005_R2_001.fastq.gz
Started analysis of Int-03_S44_L005_R1_001.fastq.gz
Started analysis of Int-03_S44_L005_R2_001.fastq.gz
Started analysis of Int-04_S45_L005_R1_001.fastq.gz
Started analysis of Int-04_S45_L005_R2_001.fastq.gz
Started analysis of Liv-01_S38_L005_R1_001.fastq.gz
Started analysis of Liv-01_S38_L005_R2_001.fastq.gz
Started analysis of Liv-02_S39_L005_R1_001.fastq.gz
Approx 5% complete for Int-01_S42_L005_R1_001.fastq.gz
Started analysis of Liv-02_S39_L005_R2_001.fastq.gz
Approx 5% complete for Int-01_S42_L005_R2_001.fas

All samples had good per base sequence quality and per sequence quality scores. The first 10bp of all samples had poor per base sequence content. Files had a length of 151bp. The samples also had poor adapter content.

### Trim reads and run fastqc

Both forward and reverse adapters were trimmed. The frist 10 base pairs were trimmed off due to poor base sequence content, likely due to the barcode. --nextseq-trim was also used due to this being run through Illumina.

In [4]:
!mkdir trimmed_reduced
!for r1 in /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/*R1_001.fastq.gz; do \
    sample=$(basename $r1 _R1_001.fastq.gz); \
    r2=${r1%%_R1_001.fastq.gz}_R2_001.fastq.gz; \
    cutadapt -j 32 -m 20 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT --nextseq-trim=10 -u 10 -U 10 -o trimmed_reduced/${sample}_R1_001.fastq.gz -p trimmed_reduced/${sample}_R2_001.fastq.gz ${r1} ${r2}; \
done

This is cutadapt 4.9 with Python 3.12.3
Command line parameters: -j 32 -m 20 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT --nextseq-trim=10 -u 10 -U 10 -o trimmed_reduced/Int-01_S42_L005_R1_001.fastq.gz -p trimmed_reduced/Int-01_S42_L005_R2_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Int-01_S42_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Int-01_S42_L005_R2_001.fastq.gz
Processing paired-end reads on 32 cores ...
Done           00:00:35    12,421,987 reads @   2.9 µs/read;  21.00 M reads/minute
Finished in 35.588 s (2.865 µs/read; 20.94 M reads/minute).

=== Summary ===

Total read pairs processed:         12,421,987
  Read 1 with adapter:               3,346,061 (26.9%)
  Read 2 with adapter:               3,263,630 (26.3%)

== Read fate breakdown ==
Pairs that were too short:             253,185 (2.0%)
Pairs written (passing filters):  

In [5]:
!mkdir trimmed_reduced/qc
!fastqc -t 32 trimmed_reduced/*.fastq.gz -o trimmed_reduced/qc

application/gzip
application/gzip
Started analysis of Int-01_S42_L005_R1_001.fastq.gz
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
Started analysis of Int-01_S42_L005_R2_001.fastq.gz
Started analysis of Int-02_S43_L005_R1_001.fastq.gz
Started analysis of Int-02_S43_L005_R2_001.fastq.gz
Started analysis of Int-03_S44_L005_R1_001.fastq.gz
Started analysis of Int-03_S44_L005_R2_001.fastq.gz
Started analysis of Int-04_S45_L005_R1_001.fastq.gz
Started analysis of Int-04_S45_L005_R2_001.fastq.gz
Started analysis of Liv-01_S38_L005_R1_001.fastq.gz
Started analysis of Liv-01_S38_L005_R2_001.fastq.gz
Started analysis of Liv-02_S39_L005_R1_001.fastq.gz
Approx 5% complete for Int-01_S42_L005_R1_001.fastq.gz
Started analysis of Liv-02_S39_L005_R2_001.fastq.gz
Approx 5% complete for Int-01_S42_L005_R2_001.fas

All samples had good per base sequence quality and per sequence quality scores. The samples had poor per base sequence content due to a sharp decrease of G percentage at the very end of the sequence. Files had a length ranging from 20bp to 141bp. The samples now have no adapter content, which is good.

### Star and Hisat Alignments

#### Build SA

In [3]:
%mkdir alignment_reduced/
%mkdir alignment_reduced/star/

!STAR \
--runThreadN 32 \
--runMode genomeGenerate \
--genomeDir genome_reduced/star \
--genomeFastaFiles genome_reduced/genome.fa \
--sjdbGTFfile genome_reduced/annotations.gtf \
--sjdbOverhang 139 \
--genomeSAindexNbases 13

	/data/users/corwinbm5021/.conda/envs/biol343_project/bin/STAR-avx2 --runThreadN 32 --runMode genomeGenerate --genomeDir genome_reduced/star --genomeFastaFiles genome_reduced/genome.fa --sjdbGTFfile genome_reduced/annotations.gtf --sjdbOverhang 139 --genomeSAindexNbases 13
	STAR version: 2.7.10b   compiled: 2023-05-25T06:56:23+0000 :/opt/conda/conda-bld/star_1684997536154/work/source
Dec 13 20:22:36 ..... started STAR run
Dec 13 20:22:36 ... starting to generate Genome files
Dec 13 20:22:42 ..... processing annotations GTF
Dec 13 20:22:45 ... starting to sort Suffix Array. This may take a long time...
Dec 13 20:22:47 ... sorting Suffix Array chunks and saving them to disk...
Dec 13 20:23:32 ... loading chunks from disk, packing SA...
Dec 13 20:23:42 ... finished generating suffix array
Dec 13 20:23:42 ... generating Suffix Array index
Dec 13 20:24:42 ... completed Suffix Array index
Dec 13 20:24:42 ..... inserting junctions into the genome indices
Dec 13 20:25:13 ... writing Genome to 

#### Star First Alignment

In [4]:
!STAR \
--runThreadN 16 \
--runMode alignReads \
--genomeDir genome_reduced/star \
--readFilesManifest manifest_reduced.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outFileNamePrefix alignment_reduced/star/

!cp alignment_reduced/star/Log.final.out alignment_reduced/star/first-pass.final.out

	/data/users/corwinbm5021/.conda/envs/biol343_project/bin/STAR-avx2 --runThreadN 16 --runMode alignReads --genomeDir genome_reduced/star --readFilesManifest manifest_reduced.tsv --readFilesCommand zcat --outSAMtype BAM SortedByCoordinate --outSAMunmapped Within --outFileNamePrefix alignment_reduced/star/
	STAR version: 2.7.10b   compiled: 2023-05-25T06:56:23+0000 :/opt/conda/conda-bld/star_1684997536154/work/source
Dec 13 20:25:20 ..... started STAR run
Dec 13 20:25:20 ..... loading genome
Dec 13 20:25:23 ..... started mapping
Dec 14 00:51:17 ..... finished mapping
Dec 14 00:51:18 ..... started sorting BAM
Dec 14 00:58:48 ..... finished successfully


#### Star Second Alignment

In [5]:
!STAR \
--runThreadN 16 \
--runMode alignReads \
--genomeDir genome_reduced/star \
--readFilesManifest manifest_reduced.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outSAMattributes NH HI AS nM RG \
--outFileNamePrefix alignment_reduced/star/ \
--sjdbFileChrStartEnd alignment_reduced/star/SJ.out.tab

!cp alignment_reduced/star/Log.final.out alignment_reduced/star/second-pass.final.out

	/data/users/corwinbm5021/.conda/envs/biol343_project/bin/STAR-avx2 --runThreadN 16 --runMode alignReads --genomeDir genome_reduced/star --readFilesManifest manifest_reduced.tsv --readFilesCommand zcat --outSAMtype BAM SortedByCoordinate --outSAMunmapped Within --outSAMattributes NH HI AS nM RG --outFileNamePrefix alignment_reduced/star/ --sjdbFileChrStartEnd alignment_reduced/star/SJ.out.tab
	STAR version: 2.7.10b   compiled: 2023-05-25T06:56:23+0000 :/opt/conda/conda-bld/star_1684997536154/work/source
Dec 14 00:58:58 ..... started STAR run
Dec 14 00:59:01 ..... loading genome
Dec 14 00:59:16 ..... inserting junctions into the genome indices
Dec 14 00:59:39 ..... started mapping
Dec 14 05:39:12 ..... finished mapping
Dec 14 05:39:13 ..... started sorting BAM
Dec 14 05:46:30 ..... finished successfully


#### Hisat Index and Alignment

In [6]:
%mkdir genome_reduced/hisat/

!hisat2_extract_splice_sites.py genome_reduced/annotations.gtf > genome_reduced/hisat/hisat_ss.tsv
!hisat2_extract_exons.py genome_reduced/annotations.gtf > genome_reduced/hisat/hisat_exons.tsv
!hisat2-build genome_reduced/genome.fa genome_reduced/hisat/genome -p 16 --ss genome_reduced/hisat/hisat_ss.tsv --exon genome_reduced/hisat/hisat_exons.tsv

Settings:
  Output files: "genome_reduced/hisat/genome.*.ht2"
  Line rate: 7 (line is 128 bytes)
  Lines per side: 1 (side is 128 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Local offset rate: 3 (one in 8)
  Local fTable chars: 6
  Local sequence length: 57344
  Local sequence overlap between two consecutive indexes: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  genome_reduced/genome.fa
Reading reference sizes
  Time reading reference sizes: 00:00:03
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:02
  Time to read SNPs and splice sites: 00:00:00
Generation 0 (391447270 -> 391447270 nodes, 0 ranks)
COUNTED NEW NODES: 0
COUNTED TEMP NODES: 0
RESIZED NODES: 2
RESIZED NODES: 0
MADE NEW NODES: 0
Generation 1 (391

In [7]:
%mkdir alignment_reduced/hisat/

!while read -r line; do \
    fq1=$(echo $line | awk '{print $1}'); \
    fq2=$(echo $line | awk '{print $2}'); \
    bn=$(basename "$fq1" _R1_001.fastq.gz); \
    rg=$(echo $line | awk '{print $3}' | sed 's/ID://' ); \
    echo "Aligning $fq1 and $fq2 with $rg"; \
    hisat2 /data/users/corwinbm5021/Sm_Mira_IvT/notebook_reduced/genome_reduced/hisat/genome -p 32 -1 $fq1 -2 $fq2 --rg-id $rg --rg SM:$rg --summary-file alignment_reduced/hisat/$bn.log --new-summary > alignment_reduced/hisat/$bn.sam; \
    done < manifest_reduced.tsv

Aligning /data/users/corwinbm5021/Sm_Mira_IvT/notebook_reduced/trimmed_reduced/Int-01_S42_L005_R1_001.fastq.gz and /data/users/corwinbm5021/Sm_Mira_IvT/notebook_reduced/trimmed_reduced/Int-01_S42_L005_R2_001.fastq.gz with Int-01
HISAT2 summary stats:
	Total pairs: 12168802
		Aligned concordantly or discordantly 0 time: 4399202 (36.15%)
		Aligned concordantly 1 time: 1100163 (9.04%)
		Aligned concordantly >1 times: 6354012 (52.22%)
		Aligned discordantly 1 time: 315425 (2.59%)
	Total unpaired reads: 8798404
		Aligned 0 time: 8444291 (95.98%)
		Aligned 1 time: 131703 (1.50%)
		Aligned >1 times: 222410 (2.53%)
	Overall alignment rate: 65.30%
Aligning /data/users/corwinbm5021/Sm_Mira_IvT/notebook_reduced/trimmed_reduced/Int-02_S43_L005_R1_001.fastq.gz and /data/users/corwinbm5021/Sm_Mira_IvT/notebook_reduced/trimmed_reduced/Int-02_S43_L005_R2_001.fastq.gz with Int-02
HISAT2 summary stats:
	Total pairs: 15058640
		Aligned concordantly or discordantly 0 time: 12201953 (81.03%)
		Aligned conc

### Convert .sam Files to .bam and Sort

In [8]:
!for sam in alignment_reduced/hisat/*.sam; do \
    bn=$(basename "$sam" .sam); \
    echo "Converting $sam to $bn.bam and sorting."; \
    samtools view -@ 32 -b $sam | samtools sort -@ 32 > alignment_reduced/hisat/$bn.bam; \
    done

Converting alignment_reduced/hisat/Int-01_S42_L005.sam to Int-01_S42_L005.bam and sorting.
[bam_sort_core] merging from 32 files and 32 in-memory blocks...
Converting alignment_reduced/hisat/Int-02_S43_L005.sam to Int-02_S43_L005.bam and sorting.
[bam_sort_core] merging from 0 files and 32 in-memory blocks...
Converting alignment_reduced/hisat/Int-03_S44_L005.sam to Int-03_S44_L005.bam and sorting.
[bam_sort_core] merging from 0 files and 32 in-memory blocks...
Converting alignment_reduced/hisat/Int-04_S45_L005.sam to Int-04_S45_L005.bam and sorting.
[bam_sort_core] merging from 0 files and 32 in-memory blocks...
Converting alignment_reduced/hisat/Liv-01_S38_L005.sam to Liv-01_S38_L005.bam and sorting.
[bam_sort_core] merging from 0 files and 32 in-memory blocks...
Converting alignment_reduced/hisat/Liv-02_S39_L005.sam to Liv-02_S39_L005.bam and sorting.
[bam_sort_core] merging from 32 files and 32 in-memory blocks...
Converting alignment_reduced/hisat/Liv-03_S40_L005.sam to Liv-03_S40

### Merge .bam Files Into One File and Generate Index

In [9]:
!samtools merge -@ 32 alignment_reduced/hisat/merged.bam  alignment_reduced/hisat/*.bam 
!samtools index -@ 32 alignment_reduced/hisat/merged.bam 

### Flagstat

In [10]:
!samtools flagstat alignment_reduced/star/Aligned.sortedByCoord.out.bam
!samtools flagstat alignment_reduced/hisat/merged.bam

252673365 + 0 in total (QC-passed reads + QC-failed reads)
3664337 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
15397899 + 0 mapped (6.09% : N/A)
249009028 + 0 paired in sequencing
124504514 + 0 read1
124504514 + 0 read2
11733514 + 0 properly paired (4.71% : N/A)
11733514 + 0 with itself and mate mapped
48 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)
573987620 + 0 in total (QC-passed reads + QC-failed reads)
324978592 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
378969909 + 0 mapped (66.02% : N/A)
249009028 + 0 paired in sequencing
124504514 + 0 read1
124504514 + 0 read2
48492300 + 0 properly paired (19.47% : N/A)
53131856 + 0 with itself and mate mapped
859461 + 0 singletons (0.35% : N/A)
59826 + 0 with mate mapped to a different chr
17776 + 0 with mate mapped to a different chr (mapQ>=5)


### Qualimap

In [11]:
!qualimap bamqc -nt 32 -outdir qualimap_reduced/star/bam -bam alignment_reduced/star/Aligned.sortedByCoord.out.bam --feature-file genome_reduced/annotations.gtf
!qualimap rnaseq -outdir qualimap_reduced/star/rnaseq -bam alignment_reduced/star/Aligned.sortedByCoord.out.bam -gtf genome_reduced/annotations.gtf
!qualimap bamqc -nt 32 -outdir qualimap_reduced/hisat/bam -bam alignment_reduced/hisat/merged.bam --feature-file genome_reduced/annotations.gtf
!qualimap rnaseq -outdir qualimap_reduced/hisat/rnaseq -bam alignment_reduced/hisat/merged.bam -gtf genome_reduced/annotations.gtf

Java memory size is set to 1200M
Launching application...

QualiMap v.2.3
Built on 2023-05-19 16:57

Selected tool: bamqc
Available memory (Mb): 35
Max memory (Mb): 1258
Starting bam qc....
Loading sam header...
Loading locator...
Loading reference...
Number of windows: 400, effective number of windows: 409
Chunk of reads size: 1000
Number of threads: 32
Initializing regions from genome_reduced/annotations.gtf.....
Found 241537 regions
Filling region references... 
Processed 50 out of 409 windows...
Processed 100 out of 409 windows...
Processed 150 out of 409 windows...
Processed 200 out of 409 windows...
Processed 250 out of 409 windows...
Processed 300 out of 409 windows...
Processed 350 out of 409 windows...
Processed 400 out of 409 windows...
Total processed windows:409
Number of reads: 249009028
Number of valid reads: 11733562
Number of correct strand reads:0

Inside of regions...
Num mapped reads: 9718104
Num mapped first of pair: 4863165
Num mapped second of pair: 4854939
Num si

### Mark Duplicates

In [1]:
%mkdir counting_reduced
%mkdir counting_reduced/logs_reduced
%mkdir counting_reduced/dedup_reduced

!picard MarkDuplicates -I alignment_reduced/star/Aligned.sortedByCoord.out.bam -M counting_reduced/logs_reduced/star_duplicates -O counting_reduced/dedup_reduced/star.bam -Xmx100g --VALIDATION_STRINGENCY SILENT
!picard MarkDuplicates -I alignment_reduced/hisat/merged.bam -M counting_reduced/logs_reduced/hisat_duplicates -O counting_reduced/dedup_reduced/hisat.bam -Xmx100g --VALIDATION_STRINGENCY SILENT

21:18:45.469 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/data/users/corwinbm5021/.conda/envs/biol343_project/share/picard-3.2.0-0/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Sat Dec 14 21:18:45 CST 2024] MarkDuplicates --INPUT alignment_reduced/star/Aligned.sortedByCoord.out.bam --OUTPUT counting_reduced/dedup_reduced/star.bam --METRICS_FILE counting_reduced/logs_reduced/star_duplicates --VALIDATION_STRINGENCY SILENT --MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP 50000 --MAX_FILE_HANDLES_FOR_READ_ENDS_MAP 8000 --SORTING_COLLECTION_SIZE_RATIO 0.25 --TAG_DUPLICATE_SET_MEMBERS false --REMOVE_SEQUENCING_DUPLICATES false --TAGGING_POLICY DontTag --CLEAR_DT true --DUPLEX_UMI false --FLOW_MODE false --FLOW_DUP_STRATEGY FLOW_QUALITY_SUM_STRATEGY --USE_END_IN_UNPAIRED_READS false --USE_UNPAIRED_CLIPPED_END false --UNPAIRED_END_UNCERTAINTY 0 --UNPAIRED_START_UNCERTAINTY 0 --FLOW_SKIP_FIRST_N_FLOWS 0 --FLOW_Q_IS_KNOWN_END false --FLOW_EFFECTIVE_QUALITY_THRESHOLD 1

### Feature Counts

In [2]:
%mkdir counting_reduced/counts_reduced

!featureCounts -T 32 \
    counting_reduced/dedup_reduced/star.bam \
    -T 32 \
    -p \
    --byReadGroup \
    -s 1 \
    --ignoreDup \
    -M \
    --fraction \
    -a genome_reduced/annotations.gtf \
    -o counting_reduced/counts_reduced/star_counts.tsv \
    --verbose

!featureCounts -T 32 \
    counting_reduced/dedup_reduced/hisat.bam \
    -T 32 \
    -p \
    --byReadGroup \
    -s 1 \
    --ignoreDup \
    -M \
    --fraction \
    -a genome_reduced/annotations.gtf \
    -o counting_reduced/counts_reduced/hisat_counts.tsv \
    --verbose


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.6

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 1 BAM file                                       ||
||                                                                            ||
||                           star.bam                                         ||
||                                                                            ||
||             Output file : star_counts.tsv                     

### Multiqc Report

In [3]:
!multiqc --force -d fastq_reduced/qc/ trimmed_reduced/qc/ alignment_reduced/ counting_reduced/ -n reports_reduced/multiqc_report.html


  /// ]8;id=974509;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.17

|           multiqc | Prepending directory to sample names
|           multiqc | Search path : /data/users/corwinbm5021/Sm_Mira_IvT/notebook_reduced/fastq_reduced/qc
|           multiqc | Search path : /data/users/corwinbm5021/Sm_Mira_IvT/notebook_reduced/trimmed_reduced/qc
|           multiqc | Search path : /data/users/corwinbm5021/Sm_Mira_IvT/notebook_reduced/alignment_reduced
|           multiqc | Search path : /data/users/corwinbm5021/Sm_Mira_IvT/notebook_reduced/counting_reduced
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 107/107  hisat/Liv-04_S41_L005.logt.tab
|    feature_counts | Found 2 reports
|            picard | Found 2 MarkDuplicates reports
|              star | Found 1 reports
|            hisat2 | Found 8 reports
|            fastqc | Found 32 reports
|           multiqc | Report      : reports_reduced/multiqc_report.html
|           multiqc | Data        : reports_reduced/mul

discuss multiqc report here

### Differential Expression

Analysis of plots generated below can be found in plots_reduced/plot_evaluations

In [1]:
%mkdir count_matrices_reduced
%mkdir plots_reduced
!Rscript diffexp_reduced.R

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    